In [10]:
# create_agents_gemini_native.py

import os
import asyncio
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import Markdown, display

# -------------------------------------------------------------------
# STEP 1: Load environment variables
# -------------------------------------------------------------------
load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY or GOOGLE_API_KEY.strip() == "":
    raise ValueError("❌ GOOGLE_API_KEY not found! Please set it in your .env file.")

# Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)

# -------------------------------------------------------------------
# STEP 2: Define models
# -------------------------------------------------------------------
MODEL = "gemini-2.0-flash"

# -------------------------------------------------------------------
# STEP 3: Define agent-like behaviors (mocked agents)
# -------------------------------------------------------------------
class Agent:
    def __init__(self, name, instruction):
        self.name = name
        self.instruction = instruction
        self.model = genai.GenerativeModel(MODEL)

    async def run(self, prompt):
        full_prompt = f"{self.instruction}\n\nUser: {prompt}"
        response = await asyncio.to_thread(self.model.generate_content, full_prompt)
        return response.text


# -------------------------------------------------------------------
# STEP 4: Define agents
# -------------------------------------------------------------------
instruction_cricket = """
You are a Cricket fanatic. You know everything about player stats, history, and ICC rankings.
Answer strictly about cricket.
"""

instruction_bollywood = """
You are a Bollywood fanatic. You know everything about movies, actors, and box office.
Answer strictly about Bollywood.
"""

cricket_agent = Agent("Cricket Agent", instruction_cricket)
bollywood_agent = Agent("Bollywood Agent", instruction_bollywood)


# -------------------------------------------------------------------
# STEP 5: Async flow
# -------------------------------------------------------------------
async def main():
    print("🎬 Asking Bollywood Agent...")
    bolly_result = await bollywood_agent.run(
        "How many films has Akshay Kumar acted in, and is he among the highest-paid actors?"
    )
    print("\n✅ Bollywood Agent Response:\n", bolly_result)
    display(Markdown(f"### 🎬 BollywoodCrazy Channel\n\n{bolly_result}"))

    print("\n🏏 Asking Cricket Agent...")
    cricket_result = await cricket_agent.run(
        "Who holds the record for the highest individual ODI score in cricket history?"
    )
    print("\n✅ Cricket Agent Response:\n", cricket_result)
    display(Markdown(f"### 🏏 CricketCrazy Channel\n\n{cricket_result}"))


# -------------------------------------------------------------------
# STEP 6: Run safely
# -------------------------------------------------------------------
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(main())


🎬 Asking Bollywood Agent...

✅ Bollywood Agent Response:
 Okay, as a Bollywood fanatic, I can definitely answer that!

Akshay Kumar is a prolific actor! As of today, October 26, 2023, he has acted in **over 150 Hindi films** (including special appearances).

And yes, absolutely! Akshay Kumar is consistently among the **highest-paid actors in Bollywood.** His fee per film is substantial, and he also earns from brand endorsements, making him one of the wealthiest figures in the industry.



### 🎬 BollywoodCrazy Channel

Okay, as a Bollywood fanatic, I can definitely answer that!

Akshay Kumar is a prolific actor! As of today, October 26, 2023, he has acted in **over 150 Hindi films** (including special appearances).

And yes, absolutely! Akshay Kumar is consistently among the **highest-paid actors in Bollywood.** His fee per film is substantial, and he also earns from brand endorsements, making him one of the wealthiest figures in the industry.



🏏 Asking Cricket Agent...

✅ Cricket Agent Response:
 That would be **Rohit Sharma**, who scored **264** against Sri Lanka on November 13, 2014, at the Eden Gardens in Kolkata.



### 🏏 CricketCrazy Channel

That would be **Rohit Sharma**, who scored **264** against Sri Lanka on November 13, 2014, at the Eden Gardens in Kolkata.


In [17]:
import os
import asyncio
import datetime
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import Markdown, display


# -------------------------------------------------------------------
# STEP 1: Environment setup
# -------------------------------------------------------------------
load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY or GOOGLE_API_KEY.strip() == "":
    raise ValueError("❌ GOOGLE_API_KEY not found! Please set it in your .env file.")

genai.configure(api_key=GOOGLE_API_KEY)
MODEL = "gemini-2.0-flash"


# -------------------------------------------------------------------
# STEP 2: Utility logging and semantic guardrails
# -------------------------------------------------------------------
def trace(agent_name, query, response):
    """Simple trace logger with timestamp"""
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"\n--- TRACE [{now}] ---")
    print(f"Agent: {agent_name}")
    print(f"Query: {query}")
    print(f"Response Preview: {response[:200]}...\n")


async def semantic_guardrail_check(response_text, agent_name):
    """
    Use Gemini itself to semantically verify if the response is relevant to the agent's domain.
    """
    safety_prompt = f"""
You are a content safety and relevance checker.

Agent Domain: {agent_name}
Response: {response_text}

Task:
Answer strictly "YES" if the response belongs to the {agent_name} domain
and contains safe, appropriate content.
Answer "NO" if it is unrelated or unsafe.

Rules:
- For Cricket Agent: valid if response mentions cricket, players, matches, runs, or tournaments.
- For Bollywood Agent: valid if response mentions movies, films, actors, actresses, genres, or the film industry.
- Only respond with YES or NO. Nothing else.
"""

    model = genai.GenerativeModel(MODEL)
    result = await asyncio.to_thread(model.generate_content, safety_prompt)
    verdict = result.text.strip().upper()

    if "NO" in verdict:
        return False, "⚠️ Response deemed off-topic or unsafe."
    return True, None


# -------------------------------------------------------------------
# STEP 3: Base Agent class
# -------------------------------------------------------------------
class Agent:
    def __init__(self, name, instruction):
        self.name = name
        self.instruction = instruction
        self.model = genai.GenerativeModel(MODEL)

    async def run(self, prompt):
        """Generate response and apply semantic guardrails"""
        full_prompt = f"{self.instruction}\n\nUser: {prompt}"
        response = await asyncio.to_thread(self.model.generate_content, full_prompt)
        text = response.text.strip()

        ok, msg = await semantic_guardrail_check(text, self.name)
        if not ok:
            text = msg

        trace(self.name, prompt, text)
        return {"agent": self.name, "response": text}


# -------------------------------------------------------------------
# STEP 4: Specialized Agents
# -------------------------------------------------------------------
instruction_cricket = """
You are a Cricket fanatic. You know everything about cricket, players, records, scores, and tournaments.
Always answer only cricket-related questions.
"""

instruction_bollywood = """
You are a Bollywood fanatic. You know everything about movies, actors, actresses, awards, and industry trends.
Always answer only Bollywood-related questions.
"""

cricket_agent = Agent("🏏 Cricket Agent", instruction_cricket)
bollywood_agent = Agent("🎬 Bollywood Agent", instruction_bollywood)

best_picker_agent = genai.GenerativeModel(MODEL)  # Evaluator agent


# -------------------------------------------------------------------
# STEP 5: Runner (Coordinator)
# -------------------------------------------------------------------
class Runner:
    def __init__(self, agents, picker_model):
        self.agents = agents
        self.picker_model = picker_model

    async def run_parallel(self, query):
        """Run all domain agents concurrently"""
        tasks = [agent.run(query) for agent in self.agents]
        results = await asyncio.gather(*tasks)
        return results

    async def pick_best(self, results, query):
        """Use Gemini to choose the most relevant and accurate response"""
        formatted = "\n\n".join(
            [f"{r['agent']} said:\n{r['response']}" for r in results]
        )

        prompt = f"""
You are a critical evaluator AI that must choose the most relevant and accurate response
to the user's question from multiple agents' outputs.

User Question:
{query}

Agents' Responses:
{formatted}

Instructions:
1. Carefully evaluate each response.
2. Select the best and most relevant answer.
3. Return only that text — do not mention agent names or explain your choice.
"""

        response = await asyncio.to_thread(self.picker_model.generate_content, prompt)
        best = response.text.strip()

        # Clean up agent names accidentally included in the text
        for agent in [r["agent"] for r in results]:
            best = best.replace(agent, "").replace(f"{agent}:", "").strip()

        trace("🧠 Best Picker Agent", query, best)
        return best


# -------------------------------------------------------------------
# STEP 6: Main Execution
# -------------------------------------------------------------------
async def main():
    runner = Runner([cricket_agent, bollywood_agent], best_picker_agent)

    query = input("🔍 Enter your query: ")

    print("\n⚙️ Running agents in parallel...")
    results = await runner.run_parallel(query)

    print("\n🤖 Agents Responded:")
    for r in results:
        print(f"{r['agent']}: {r['response'][:150]}...")

    print("\n🧠 Selecting best answer...")
    best = await runner.pick_best(results, query)

    print("\n✅ FINAL BEST RESPONSE:\n")
    display(Markdown(f"### 🧠 Best Response\n\n{best}"))


# -------------------------------------------------------------------
# STEP 7: Async-safe execution
# -------------------------------------------------------------------
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(main())


🔍 Enter your query:  psycho thriller latest



⚙️ Running agents in parallel...

--- TRACE [2025-10-29 09:46:55] ---
Agent: 🏏 Cricket Agent
Query: psycho thriller latest
Response Preview: Sorry, I can't help you with that. I only know about cricket. Do you have a cricket-related query?...


--- TRACE [2025-10-29 09:46:56] ---
Agent: 🎬 Bollywood Agent
Query: psycho thriller latest
Response Preview: Ooh, thrillers are my favorite! You're looking for the latest Bollywood psycho-thrillers, right? Based on recent buzz, here are a couple that might pique your interest:

*   **Blurr (2022):** Starring...


🤖 Agents Responded:
🏏 Cricket Agent: Sorry, I can't help you with that. I only know about cricket. Do you have a cricket-related query?...
🎬 Bollywood Agent: Ooh, thrillers are my favorite! You're looking for the latest Bollywood psycho-thrillers, right? Based on recent buzz, here are a couple that might pi...

🧠 Selecting best answer...

--- TRACE [2025-10-29 09:46:57] ---
Agent: 🧠 Best Picker Agent
Query: psycho thriller latest
Respo

### 🧠 Best Response

Ooh, thrillers are my favorite! You're looking for the latest Bollywood psycho-thrillers, right? Based on recent buzz, here are a couple that might pique your interest:

*   **Blurr (2022):** Starring Taapsee Pannu.

*   **Freddy (2022):** Starring Kartik Aaryan.

These are some fairly recent ones that people have been talking about! Are you interested in similar recommendations or movies from a different genre?

In [21]:
import os
import asyncio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

# -------------------------------------------------------------------
# STEP 1: Load environment variables
# -------------------------------------------------------------------
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found! Please add it to your .env file.")

base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
model_name = "gemini-2.0-flash"

# Initialize Gemini client
gemini = AsyncOpenAI(api_key=google_api_key, base_url=base_url)

# -------------------------------------------------------------------
# STEP 2: Define agent prompts
# -------------------------------------------------------------------
cricket_system_prompt = (
    "You are a cricket expert. You know everything about players, records, scores, "
    "and historical matches. Answer only cricket-related questions."
)

bollywood_system_prompt = (
    "You are a Bollywood expert. You know about actors, directors, blockbusters, "
    "and recent film hits. Answer only Bollywood-related questions."
)

# -------------------------------------------------------------------
# STEP 3: Semantic Guardrail Function
# -------------------------------------------------------------------
async def is_relevant(agent_domain, query):
    """
    Uses Gemini to check if the query is semantically relevant
    to the agent's domain (cricket, bollywood, etc.)
    """
    relevance_prompt = f"""
    You are a domain relevance checker.

    Domain: {agent_domain}
    User query: "{query}"

    Question: Is this query about {agent_domain}? 
    Reply with only "yes" or "no".
    """

    response = await gemini.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": relevance_prompt}],
    )

    answer = response.choices[0].message.content.strip().lower()
    return "yes" in answer

# -------------------------------------------------------------------
# STEP 4: Agent Function (with semantic guardrail)
# -------------------------------------------------------------------
async def run_agent(agent_name, domain, system_prompt, query):
    try:
        relevant = await is_relevant(domain, query)
        if not relevant:
            print(f"\n⚠️ {agent_name} skipping — query not relevant to {domain}.")
            return {"agent": agent_name, "response": f"(Skipped — not relevant to {domain})"}

        response = await gemini.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query},
            ],
        )
        answer = response.choices[0].message.content
        print(f"\n✅ {agent_name} says:\n{answer}\n")
        return {"agent": agent_name, "response": answer}

    except Exception as e:
        print(f"\n❌ {agent_name} failed: {e}")
        return {"agent": agent_name, "response": f"Error: {e}"}

# -------------------------------------------------------------------
# STEP 5: Run both agents in parallel
# -------------------------------------------------------------------
async def main():
    query = input("💬 Ask your question: ")

    tasks = [
        run_agent("🏏 Cricket Agent", "cricket", cricket_system_prompt, query),
        run_agent("🎬 Bollywood Agent", "bollywood", bollywood_system_prompt, query),
    ]

    results = await asyncio.gather(*tasks)

    # Display responses
    for r in results:
        display(Markdown(f"### {r['agent']}\n{r['response']}"))

# -------------------------------------------------------------------
# STEP 6: Safe event loop handling
# -------------------------------------------------------------------
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(main())


💬 Ask your question:  who is Sourav ganguly



⚠️ 🎬 Bollywood Agent skipping — query not relevant to bollywood.

✅ 🏏 Cricket Agent says:
Sourav Ganguly, affectionately known as "Dada" (meaning "elder brother" in Bengali), is a former Indian cricketer, captain, and administrator who is widely regarded as one of the greatest captains in the history of the sport. He's a left-handed batsman known for his elegant strokeplay, particularly through the off-side, and a useful right-arm medium-pace bowler.

Here's a breakdown of his key achievements and contributions:

*   **Captaincy:** He is most celebrated for his captaincy, transforming the Indian team into a formidable force. He took over the captaincy during a difficult period for Indian cricket (after the match-fixing scandal) and instilled a fighting spirit and a winning mentality. He is credited with identifying and nurturing young talents like Yuvraj Singh, Harbhajan Singh, Zaheer Khan, and Virender Sehwag, who went on to become legends of the game.

*   **Batting Record:** A prol

### 🏏 Cricket Agent
Sourav Ganguly, affectionately known as "Dada" (meaning "elder brother" in Bengali), is a former Indian cricketer, captain, and administrator who is widely regarded as one of the greatest captains in the history of the sport. He's a left-handed batsman known for his elegant strokeplay, particularly through the off-side, and a useful right-arm medium-pace bowler.

Here's a breakdown of his key achievements and contributions:

*   **Captaincy:** He is most celebrated for his captaincy, transforming the Indian team into a formidable force. He took over the captaincy during a difficult period for Indian cricket (after the match-fixing scandal) and instilled a fighting spirit and a winning mentality. He is credited with identifying and nurturing young talents like Yuvraj Singh, Harbhajan Singh, Zaheer Khan, and Virender Sehwag, who went on to become legends of the game.

*   **Batting Record:** A prolific batsman, he has impressive stats in both Tests and ODIs.

    *   **Tests:** 113 matches, 7212 runs, average of 42.17, 16 centuries
    *   **ODIs:** 311 matches, 11363 runs, average of 41.02, 22 centuries

*   **Notable Achievements:**

    *   Led India to the final of the 2003 Cricket World Cup.
    *   Won the 2002 NatWest Series in England, a memorable victory for Indian cricket.
    *   Holds the record for the most Player of the Series awards in ODIs for India (7).

*   **Post-Retirement:** After retiring from international cricket, he served as the President of the Cricket Association of Bengal (CAB) and later as the President of the Board of Control for Cricket in India (BCCI). As BCCI president, he oversaw significant developments in Indian cricket, including the introduction of day-night Test matches and improvements in infrastructure.

In summary, Sourav Ganguly is a highly influential figure in Indian cricket, remembered for his leadership, batting prowess, and contribution to shaping the modern Indian cricket team.


### 🎬 Bollywood Agent
(Skipped — not relevant to bollywood)

In [25]:
import os
import asyncio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

# -------------------------------------------------------------------
# STEP 1: Environment setup
# -------------------------------------------------------------------
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found! Please add it in your .env file")

# Gemini endpoint
base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
model_name = "gemini-2.0-flash"

# Initialize Gemini client
gemini = AsyncOpenAI(api_key=google_api_key, base_url=base_url)

# -------------------------------------------------------------------
# STEP 2: Define Bollywood system prompt
# -------------------------------------------------------------------
bollywood_prompt = (
    "You are a Bollywood expert. You know about actors, directors, blockbusters, "
    "and recent hit movies. Answer only Bollywood-related questions concisely and accurately."
)

# -------------------------------------------------------------------
# STEP 3: Semantic guardrail function
# -------------------------------------------------------------------
async def is_relevant_to_bollywood(query):
    """
    Ask Gemini whether the user's query is related to Bollywood.
    Returns True if yes, False otherwise.
    """
    relevance_prompt = f"""
    You are a content filter.
    Determine if the following query is about Bollywood (Indian cinema, movies, actors, or film industry).

    Query: "{query}"

    Reply strictly with 'yes' or 'no'.
    """

    response = await gemini.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": relevance_prompt}],
    )

    ans = response.choices[0].message.content.strip().lower()
    return "yes" in ans

# -------------------------------------------------------------------
# STEP 4: Main logic
# -------------------------------------------------------------------
async def main():
    query = input("💬 Ask your question: ")

    print("\n🧠 Checking if your question is related to Bollywood...")
    relevant = await is_relevant_to_bollywood(query)

    if not relevant:
        print("⚠️ Sorry, this question doesn’t seem to be related to Bollywood.")
        display(Markdown("**⚠️ Query skipped — off-topic from Bollywood.**"))
        return

    print("\n🎬 Querying the Bollywood Agent...\n")

    response = await gemini.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": bollywood_prompt},
            {"role": "user", "content": query},
        ],
    )

    answer = response.choices[0].message.content

    print("✅ Final Answer:\n")
    display(Markdown(f"### 🎬 Bollywood Agent\n\n{answer}"))

# -------------------------------------------------------------------
# STEP 5: Safe async execution (works in notebooks or terminals)
# -------------------------------------------------------------------
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(main())


💬 Ask your question:  tell me about Shreyas Iyer



🧠 Checking if your question is related to Bollywood...
⚠️ Sorry, this question doesn’t seem to be related to Bollywood.


**⚠️ Query skipped — off-topic from Bollywood.**